In [ ]:
from pathlib import Path
import os
from dotenv import load_dotenv, find_dotenv

# .env dosyasını klasör ağacında yukarı doğru arayıp yükle
env_path = find_dotenv()  # kökten/üst klasörlerden otomatik arar
loaded = load_dotenv(env_path)
print("Loaded .env from:", env_path if loaded else "NOT FOUND")

POLYGON_API_KEY = os.getenv("POLYGON_API_KEY")
FRED_API_KEY    = os.getenv("FRED_API_KEY")

# hızlı test
assert POLYGON_API_KEY and len(POLYGON_API_KEY) > 5, "POLYGON_API_KEY not found"
print("Polygon key loaded ✔")


In [ ]:
import yfinance as yf
import pandas as pd
# 1) fetch today’s S&P 500 close
ticker      = yf.Ticker("^GSPC")
hist        = ticker.history(period="1d")
underlying_date  = hist.index[0].date()
underlying_close = hist['Close'].iloc[0]   # use iloc to avoid FutureWarning

In [ ]:
from polygon import RESTClient
from datetime import datetime, timedelta, timezone

# ── API client ──────────────────────────────
client = RESTClient(POLYGON_API_KEY)

# ── Define variables ─────────────────────────
strike_low = underlying_close*0.95    # example: lower bound for strike
strike_high =underlying_close*1.05   # example: upper bound for strike

beginning_days =5  # start from today + 30 days
end_days =30# end_date = beginning_date + 200 days

Ticker="I:SPX"
today = datetime.now(timezone.utc).date()
beginning_date = (today + timedelta(days=beginning_days)).isoformat()
end_date = (today + timedelta(days=beginning_days + end_days)).isoformat()

# ── Get options chain with filters ───────────
options_chain = []
for o in client.list_snapshot_options_chain(
    Ticker,  # underlying ticker
    params={
        "order": "asc",
        "limit": 50,  # max contracts per page
        "sort": "ticker",
        "strike_price.gte": strike_low,
        "strike_price.lte": strike_high,
        "expiration_date.gte": beginning_date,
        "expiration_date.lte": end_date,
    },
):
    options_chain.append(o)

import pandas as pd

# assume options_chain is your list of OptionContractSnapshot objects
records = []
for o in options_chain:
    rec = {
        # contract details
        'ticker':                  o.details.ticker,
        'contract_type':           o.details.contract_type,
        'exercise_style':          o.details.exercise_style,
        'expiration_date':         o.details.expiration_date,
        'strike_price':            o.details.strike_price,
        'shares_per_contract':     o.details.shares_per_contract,
        # snapshot-level fields
        'break_even_price':        o.break_even_price,
        'implied_volatility':      o.implied_volatility,
        'open_interest':           o.open_interest,
        'fair_market_value':       o.fair_market_value,
        # day session stats
        'day_open':                o.day.open,
        'day_high':                o.day.high,
        'day_low':                 o.day.low,
        'day_close':               o.day.close,
        'day_previous_close':      o.day.previous_close,
        'day_change':              o.day.change,
        'day_change_percent':      o.day.change_percent,
        'day_volume':              o.day.volume,
        'day_vwap':                o.day.vwap,
        # greeks
        'delta':                   o.greeks.delta,
        'gamma':                   o.greeks.gamma,
        'theta':                   o.greeks.theta,
        'vega':                    o.greeks.vega,
        # underlying asset
        'underlying_ticker':       o.underlying_asset.ticker,
        'underlying_price':        o.underlying_asset.price,
        'underlying_change_to_bep':o.underlying_asset.change_to_break_even,
        # you can add last_quote / last_trade if you want
    }
    records.append(rec)

df = pd.DataFrame(records)

# — your existing options df
# df = pd.DataFrame(records)  # however you built it

# 2) broadcast to your options df
df['underlying_date']  = underlying_date
df['underlying_close'] = underlying_close



In [ ]:
df.head

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from scipy.interpolate import griddata

# 1) make sure your date columns are datetimes
df['expiration_date'] = pd.to_datetime(df['expiration_date'])
df['underlying_date' ] = pd.to_datetime(df['underlying_date'])

# 2) compute DTE in days
df['DTE'] = (df['expiration_date'] - df['underlying_date']).dt.days

# 3) drop any rows missing IV/strike/DTE
df_iv = df.dropna(subset=['implied_volatility','strike_price','DTE'])

# 4) build a regular grid
strike_vals = np.linspace(df_iv['strike_price'].min(),
                          df_iv['strike_price'].max(), 100)
dte_vals    = np.linspace(df_iv['DTE'].min(),
                          df_iv['DTE'].max(),        100)
strike_grid, dte_grid = np.meshgrid(strike_vals, dte_vals)

# 5) interpolate the raw IVs onto that grid
iv_grid = griddata(
    points=(df_iv['strike_price'], df_iv['DTE']),
    values=df_iv['implied_volatility'],
    xi=(strike_grid, dte_grid),
    method='cubic'
)

# 6) plot
fig = plt.figure(figsize=(12,8))
ax  = fig.add_subplot(111, projection='3d')
ax.plot_surface(strike_grid, dte_grid, iv_grid,
                linewidth=0.2, antialiased=True)

ax.set_title("Implied Volatility Surface")
ax.set_xlabel("Strike Price")
ax.set_ylabel("Days to Expiration")
ax.set_zlabel("Implied Volatility")

plt.show()




In [ ]:
import matplotlib.pyplot as plt

# assume strike_grid, dte_grid, iv_grid already exist from your interpolation step

fig, ax = plt.subplots(figsize=(10,6))

# 1) filled contours
cf = ax.contourf(
    strike_grid,
    dte_grid,
    iv_grid,
    levels=20,        # number of filled “bands”
    cmap='viridis',   # colormap (you can omit to get default)
    extend='both'     # extend colorbar at ends if values outside range
)

# 2) add contour lines on top
cs = ax.contour(
    strike_grid,
    dte_grid,
    iv_grid,
    levels=10,        # number of lines
    colors='k',       # black lines
    linewidths=0.5
)

# 3) annotate the contour lines (optional)
ax.clabel(cs, fmt='%.2f', fontsize=8)

# 4) colorbar and labels
cbar = fig.colorbar(cf, ax=ax)
cbar.set_label('Implied Volatility')

ax.set_title("I:SPX Implied Volatility Contour Map")
ax.set_xlabel("Strike Price")
ax.set_ylabel("Days to Expiration (DTE)")

plt.tight_layout()
plt.show()


In [ ]:
import numpy as np
import pandas as pd
from scipy.optimize import least_squares
import matplotlib.pyplot as plt

# 0) spot price
S = float(df['underlying_close'].iloc[0])

# 1) helper: SVI total-variance function
def svi_w(pars, k):
    a, b, rho, m, sig = pars
    return a + b*( rho*(k-m) + np.sqrt((k-m)**2 + sig**2) )

# 2) calibrator for one slice
def fit_svi_slice(k, w):
    # initial guess & bounds
    x0     = np.array([w.mean(), 0.1, 0.0, 0.0, 0.1])
    bounds = ([-np.inf,   0.0, -0.999, -np.inf, 1e-6],
              [ np.inf, np.inf,  0.999,  np.inf, np.inf])
    res = least_squares(lambda p: svi_w(p, k) - w,
                        x0, bounds=bounds)
    return res.x

# 3) loop over expiries
svi_params = {}
for T, g in df.groupby('DTE'):
    sub = g.dropna(subset=['strike_price','implied_volatility'])
    if len(sub) < 6: continue
    
    k = np.log(sub['strike_price']/S).values
    w = (sub['implied_volatility']**2)*(T/252.0)
    
    params = fit_svi_slice(k, w)
    svi_params[T] = params

# turn into DataFrame
params = pd.DataFrame.from_dict(svi_params, orient='index',
                                columns=['a','b','rho','m','sigma'])
params.index.name = 'DTE'
params = params.sort_index()

# 4) plot raw vs. SVI for each slice
fig, ax = plt.subplots(figsize=(10,6))
for T, p in svi_params.items():
    if T<10:
    # raw
        sub = df[df['DTE']==T].dropna(subset=['strike_price','implied_volatility'])
        ax.scatter(sub['strike_price'], sub['implied_volatility'],
                   s=10, alpha=0.3, label=f"{T}d raw")
        # fitted
        ks = np.linspace(min(sub['strike_price']), max(sub['strike_price']), 100)
        k = np.log(ks/S)
        w = svi_w(p, k)
        iv = np.sqrt(w / (T/252.0))
        ax.plot(ks, iv, linewidth=2, label=f"{T}d SVI")
        
ax.set_xlabel("Strike Price")
ax.set_ylabel("Implied Volatility")
ax.set_title("Raw IV vs. SVI Fit by Expiry")
ax.legend(ncol=2, fontsize=8)
plt.tight_layout()
plt.show()

# 5) check calendar arbitrage on a k-grid
#    build a small grid in k & T
DTEs = np.array(sorted(svi_params.keys()))
Ks   = np.linspace(df['strike_price'].min(), df['strike_price'].max(), 50)
kgrid = np.log(Ks/S)
Tgrid = DTEs/252.0

# compute w_grid[T_idx, k_idx]
w_grid = np.zeros((len(DTEs), len(Ks)))
for i, T in enumerate(DTEs):
    p = svi_params[T]
    w_grid[i] = svi_w(p, kgrid)

# look for any w(T[i+1],k) < w(T[i],k)
bad = []
for i in range(len(DTEs)-1):
    diff = w_grid[i+1] - w_grid[i]
    if np.any(diff < -1e-8):
        bad.append((DTEs[i], DTEs[i+1]))

if bad:
    print("Calendar arbitrage detected between maturities:")
    for t1, t2 in bad:
        print(f"  {t1}d → {t2}d")
else:
    print("No calendar arbitrage detected on the tested grid.")


In [ ]:
import numpy as np

# 1) build a small (T,k) grid from your calibrated slices
DTEs = np.array(sorted(params.index))           # your fitted expiries
Ks   = np.linspace(df['strike_price'].min(), df['strike_price'].max(), 50)
kgrid = np.log(Ks/S)

# 2) populate total‐variance grid
w_grid = np.zeros((len(DTEs), len(Ks)))
for i, T in enumerate(DTEs):
    a,b,rho,m,sig = params.loc[T]
    w_grid[i] = a + b*( rho*(kgrid-m) + np.sqrt((kgrid-m)**2 + sig**2) )

# 3) look for any decrease in T
violations = []
for i in range(len(DTEs)-1):
    diff = w_grid[i+1] - w_grid[i]
    if np.any(diff < -1e-8):
        violations.append((DTEs[i], DTEs[i+1]))

if violations:
    print("⚠️ Calendar arbitrage between maturities:")
    for t1, t2 in violations:
        print(f"   {t1}d → {t2}d (some w(T2,K) < w(T1,K))")
else:
    print("✅ No calendar arbitrage detected on the tested grid.")


In [ ]:
# make a copy to avoid SettingWithCopyWarning
df2 = df[df['DTE'].isin(params.index)].copy()

# prepare the new column
df2['fitted_IV'] = np.nan

S = float(df2['underlying_close'].iloc[0])

# for each calibrated slice, compute & assign
for T, (a,b,rho,m,sig) in params.iterrows():
    mask = df2['DTE'] == T
    strikes = df2.loc[mask, 'strike_price']
    k       = np.log(strikes / S)
    w       = a + b*(rho*(k - m) + np.sqrt((k - m)**2 + sig**2))
    iv_fit  = np.sqrt(w / (T/252.0))
    
    df2.loc[mask, 'fitted_IV'] = iv_fit

# now df2[['DTE','strike_price','implied_volatility','fitted_IV']] lines up
print(df2[['DTE','strike_price','implied_volatility','fitted_IV']].head())


In [ ]:
import numpy as np

# assume df2 is your DataFrame with columns:
#  ['DTE','strike_price','fitted_IV','underlying_close']

S = float(df2['underlying_close'].iloc[0])  # spot

tol = 1e-8
print("Butterfly arbitrage check per expiry (∂²w/∂k² ≥ 0?):")

for T in sorted(df2['DTE'].unique()):
    sub = df2[(df2['DTE']==T) & df2['fitted_IV'].notnull()]
    if len(sub) < 3:
        print(f"  {T}d: not enough strikes to test")
        continue

    # sort by strike
    strikes = sub['strike_price'].values
    iv      = sub['fitted_IV'].values
    idx     = np.argsort(strikes)
    K       = strikes[idx]
    iv      = iv[idx]

    # log‐moneyness and total variance
    k = np.log(K / S)
    T_years = T / 252.0
    w = iv**2 * T_years

    # second derivative wrt k
    dw_dk  = np.gradient(w, k)
    d2w_dk2 = np.gradient(dw_dk, k)

    if np.any(d2w_dk2 < -tol):
        print(f"  ⚠️ {T}d: NON‐CONVEX w(k) detected (butterfly arb)")
    else:
        print(f"  ✅ {T}d: w(k) is convex")
